In [ ]:
%%time 

# matrix of best errors for differen k nearest neighbors

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from scipy import spatial
from datetime import timedelta
import seaborn as sns

import sys
import collections
import itertools
from scipy.stats import mode


from numpy import *
from pylab import *
from scipy.spatial import distance

import scipy 
from sklearn import metrics
from datetime import timedelta

    
    
# sliding window

def GetShiftingWindows(thelist, size):
    return [ thelist[x:x+size] for x in range( len(thelist) - size + 1 ) ]



def CalcEuclDist(dfspeed, speed, index, time_horizon):
    #####################
    # speed = df.speed.values
    # time_horizon = 24   length
    # index of a the test series
    #####################

    
    X_test = dfspeed.ix[index:index+time_horizon].values 
    #X_test = df.speed.iloc[index:index+time_horizon].values 
    X_train = GetShiftingWindows(speed, time_horizon)

    X_test = np.array(X_test)
    X_train = np.array(X_train)
    
    #m = KnnDtw(n_neighbors=no_neighbors)

    distances = []

    # remove the WHOLE [includinng subseries ] series from the training set
    # chk indexes of similar series
    for i in range(0, len(X_train)):   
        dist  = scipy.spatial.distance.euclidean(X_train[i], X_test)
        distances.append(dist)
    
    return distances 



#error definition
def mean_absolute_percentage_error_value(y_true, y_pred):
        to_mean = []
        for true, pred in zip(y_true, y_pred):
            if true != 0:
                to_mean.append(np.abs((true - pred) / true))
        return np.mean(to_mean) * 100

def mae( y_true, y_pred):
        return metrics.mean_absolute_error(y_true, y_pred)

def rmse( y_true, y_pred):
        return np.sqrt(metrics.mean_squared_error(y_true, y_pred))


    
data_resol = [ '15min']
    
# data_resol = ['1h', '30min', '15min']
pred_horizons = [24, 24, 24]

#len of df: 3153

#indexes = np.arange(10, 2902, 166)


# TODO remove April from the first list of indexes and fix others too
# indexes for 1h, 30m, 15min

indexes = np.arange(10, 11433, 120)


# i1 = np.arange(10, 2902, 166)
# i2 = np.arange(10, 5756, 122)
# i3 = np.arange(10, 11433, 120)
# indexes = [i1, i2, i3]

# indexes = [ np.arange(10, 2902, 166),
#         np.arange(10, 5756, 122), 
#          np.arange(10, 11433, 120)  ] 

# indexes = [[10,  176,  342,  508,  674,  840, 1006, 1172, 1338, 1504, 1670,  1836, 2002, 2168, 2334, 2500, 2666, 2832],
           
# [ 10,  132,  254,  376,  498,  620,  742,  864,  986, 1108, 1230,  1352, 1474, 1596, 1718, 1840, 1962, 2084, 2206, 2328, 2450, 2572,
#        2694, 2816, 2938, 3060, 3182, 3304, 3426, 3548, 3670, 3792, 3914,      4036, 4158, 4280, 4402, 4524, 4646, 4768, 4890, 5012, 5134, 5256,
#        5378, 5500, 5622, 5744], 
           
# [ 10,   130,   250,   370,   490,   610,   730,   850,   970,     1090,  1210,  1330,  1450,  1570,  1690,  1810,  1930,  2050, 
#  2170,  2290,  2410,  2530,  2650,  2770,  2890,  3010,  3130,    3250,  3370,  3490,  3610,  3730,  3850,  3970,  4090,  4210,
#         4330,  4450,  4570,  4690,  4810,  4930,  5050,  5170,  5290,     5410,  5530,  5650,  5770,  5890,  6010,  6130,  6250,
#       6370, 6490,  6610,  6730,  6850,  6970,  7090,  7210,  7330,  7450,    7570,  7690,  7810,  7930,  8050,  8170,  8290,  
#      8410,  8530, 8650,  8770,  8890,  9010,  9130,  9250,  9370,  9490,  9610,      9730,  9850,  9970, 10090, 10210, 10330, 
#      10450, 10570, 10690, 10810, 10930, 11050, 11170, 11290, 11410]]

time_freq = [ '15m']

#time_freq = ['h', '30m', '15m']
no_neigbors = np.arange(1,31,1)


for res, hor, idxs, tf in zip(data_resol, pred_horizons, indexes, time_freq): 
    print('************************** data resolution [{}] ******************************************'.format(res))
    
    df = pd.read_csv('D:\data from MnM Samdal raw\Samdal_power_{}_no_outliers.csv'.format(res), parse_dates=[0])




    #df.precipitation = df.precipitation.fillna(value=0)
    df.power = df.power.interpolate(method='linear', limit = 2 )
    df.speed = df.speed.interpolate(method='linear', limit = 2 )

    df = df.rename( columns={"Unnamed: 0": "time"})


    # discard April
    df = df.drop(df[df.time.dt.month==4].index)

    df.time = pd.to_datetime(df.time, format='%Y%m%d %H:%M:%s')
    df = df.set_index('time')

    df = df.dropna(subset=['speed'])

    max_power =np.max(df.power)
    speed = df.speed.values



    time_horizon_lengths = np.arange(1,hor+1,1)

    no_neigbors = np.arange(1,31,1)

    list_nMAE_for_horizon_mtrx =[]
    list_nRMSE_for_horizon_mtrx = []


    list_last_pred_values = []

    for t in time_horizon_lengths:
        print('time horizon [{}]: '.format(res), t, '\n')

        list_nMAE_for_horizon = []
        list_nRMSE_for_horizon = [] 


                # read index values of top neighbors to find them in df
                #top_neigh_indexes = sorted_neigh.iloc[1:k+1].index    # take starting from ix 1 becauxe ix 0 is the same series

        for k in no_neigbors:
            #print('no_neighbors= ', k , '\n')

            list_nMAE = []
            list_nRMSE = []


            for u in indexes:


                distances = CalcEuclDist(df.speed, speed, u, t)


                df = df[:len(distances)]
                df['knnDTW_distances'] = distances

                # TODO sort acc to two distances?
                # no? use knnDTW_distances instead of euclidean ! ?

                # TODO remove indexes close to the test series
                sorted_neigh = df.sort(columns=['knnDTW_distances'], ascending=True)
            #        print('sorted_neigh', sorted_neigh.ix[:, ['power', 'knnDTW_distances']].head(5))

                ##########################

                # k neighbors
                top_neigh_indexes = sorted_neigh.iloc[:k+1].index    # +1 because the first one is true series
                #top_neigh_indexes = sorted_neigh.iloc[:30].index    # +1 because the first one is true series

            #        print('top_neigh_indexes', top_neigh_indexes)
                # find 6h windows based on indexes

                time_horizon = np.timedelta64((t.item()-1), tf) # subtract 1 because we start from i, so i+5=6

                found_6h_series = []

                for m in top_neigh_indexes:
                #    power_series = df.ix[i:i+6,'power']
                    power_series = df.ix[m:m+time_horizon,'power'].values
                    found_6h_series.append(power_series)   # first index is the tested series [true values]

                #print(df_powers, '\n')    

                df_powers = pd.DataFrame(data=found_6h_series) # rows are neigboring series, #calc avg for columns
                #print(df_powers, '\n')

                true_power_values = df_powers.iloc[0,:]
            #    print('tru', true_power_values, '\n')

                predicted_power = df_powers.iloc[1:,:].mean(axis=0)
                #print('predict', predicted_power, '\n')
            #    print('predict ty', type(predicted_power), '\n')

                last_predicted_power = df_powers.iloc[1:,-1].mean(axis=0)
                #print('last_predict', last_predicted_power, '\n')


            #         predicted_power = df_powers.iloc[1:,:]
            #         print('predict', predicted_power, '\n')

            # TODO : deal with nans in true power series
            # chk interpolation instead of ffill
                            # protect agains NaN
    #             if np.any(np.isnan(true_power_values))==True:
    #                 true_power_values = true_power_values.fillna(method='ffill' )

                #print('tru', true_power_values, '\n')

            ############################
            # finding similar speeds

                found_speed_series = []

                for m in top_neigh_indexes:
                #    power_series = df.ix[i:i+6,'power']
                    speed_series = df.ix[m:m+time_horizon,'speed'].values
                    found_speed_series.append(speed_series)   # first index is the tested series [true values]

                #print(df_powers, '\n')    

                df_speeds = pd.DataFrame(data=found_speed_series) # rows are neigboring series, #calc avg for columns
                #print(df_powers, '\n')

                true_speed_values = df_speeds.iloc[0,:]
            #    print('tru', true_power_values, '\n')

                predicted_speed = df_speeds.iloc[1:,:].mean(axis=0)
                #print('predict speed', predicted_speed, '\n')
            #    print('predict ty', type(predicted_power), '\n')

                last_predicted_speed = df_speeds.iloc[1:,-1].mean(axis=0)
                #print('last_predict speed', last_predicted_speed, '\n')


            #         predicted_power = df_powers.iloc[1:,:]
            #         print('predict', predicted_power, '\n')

            # TODO : deal with nans in true power series
            # chk interpolation instead of ffill
                            # protect agains NaN
    #             if np.any(np.isnan(true_speed_values))==True:
    #                 true_speed_values = true_speed_values.fillna(method='ffill' )

                #print('tru speed', true_speed_values, '\n')

                if np.any(np.isnan(true_power_values))==True:
                    break

                if np.any(np.isnan(predicted_power))==True:
                    break

            ###########

                MAE_1h = mae(true_power_values, predicted_power)    # it's already an avg because i took a mean of predicted power
                RMSE_1h = rmse(true_power_values, predicted_power)
            #        print('MAE_1h', MAE_1h, '\n')


                nMAE_1h = (MAE_1h/max_power)*100
                nRMSE_1h = (RMSE_1h/max_power)*100
            #        print('nMAE_1h', nMAE_1h, '\n')

                list_nMAE.append(nMAE_1h)

            avg_nMAE = sum(list_nMAE)/(len(list_nMAE))
            avg_nRMSE = sum(list_nRMSE)/(len(list_nRMSE))

            #print('avg errors for ', k , ' neighbors: \n' ,'avg_nMAE = ' , nMAE_1h )

            list_nMAE_for_horizon.append(avg_nMAE)
            list_nRMSE_for_horizon.append(avg_nRMSE)
            
            #print(len(list_nMAE_for_horizon))
            
#             # save results to columns
#             csv_input = pd.read_csv('kNN_Euc_k_matrix.csv')
#             csv_input['nMAE_{}_hor_{}'.format(res,t)] = list_nMAE_for_horizon
#             csv_input['nRMSE_{}_hor_{}'.format(res,t)] = list_nRMSE_for_horizon

#             csv_input.to_csv('kNN_Euc_k_matrix.csv', index=False)



        #print('list_nMAE_for_horizon ', t, 'list: ', list_nMAE_for_horizon)
        list_last_pred_values.append(last_predicted_power)


        list_nMAE_for_horizon_mtrx.append(list_nMAE_for_horizon)
        list_nRMSE_for_horizon_mtrx.append(list_nRMSE_for_horizon)

    # TODO take last values from the predicted series     
    #print('predicted series', list_last_pred_values)


    csv_input = pd.read_excel('kNN_Euc_k_matrix.xlsx')
    csv_input['nMAE_{}_hor_{}'.format(res,t)] = list_nMAE_for_horizon_mtrx
    csv_input['nRMSE_{}_hor_{}'.format(res,t)] = list_nRMSE_for_horizon_mtrx

    csv_input.to_excel('kNN_Euc_k_matrix.xlsx', index=False)